# Neural Network Recommendation Systems

## NeuralMF

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, regularizers

2025-10-06 16:23:21.736728: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-06 16:23:21.783355: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-06 16:23:22.514396: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/anubh/miniconda3/envs/dl-env/lib/python3.11/site-packages/tensorflow/python/keras/engine/training_arrays_v1.py:37: UserWarning: A NumPy version >=1.25.2 and <2.6.0 is required for this version of SciPy (detected version 1.24.3)
  fr

In [88]:
df_rating = pd.read_csv("rating.csv")
df_movies = pd.read_csv("movie.csv")    

In [5]:
time_stamp = pd.to_datetime(df_rating['timestamp'])

In [6]:
df_rating['year'] = time_stamp.dt.year

In [7]:
df_rating.groupby('year').count()

,userId,movieId,rating,timestamp
year,,,,
1995,4,4,4,4
1996,1612609,1612609,1612609,1612609
1997,700982,700982,700982,700982
1998,308070,308070,308070,308070
1999,1198384,1198384,1198384,1198384
2000,1953659,1953659,1953659,1953659
2001,1186125,1186125,1186125,1186125
2002,869719,869719,869719,869719
2003,1035878,1035878,1035878,1035878


In [95]:
# Dense 0..n_users-1 for all users in ratings
user_codes, user_uniqs = pd.factorize(df_rating["userId"], sort=True)
# Dense 0..n_items-1 for all items in ratings
item_codes, item_uniqs = pd.factorize(df_rating["movieId"], sort=True)

df = df_rating.copy()
df["user_idx"]  = user_codes.astype(np.int32)
df["movie_idx"] = item_codes.astype(np.int32)

n_users = int(user_codes.max()) + 1
n_items = int(item_codes.max()) + 1
print("n_users:", n_users, "n_items:", n_items)
# Sanity: dense
assert df["movie_idx"].nunique() == n_items
assert df["movie_idx"].min() == 0 and df["movie_idx"].max() == n_items - 1


movieId_by_index = np.asarray(item_uniqs) 

movies_map = df_movies.set_index("movieId")["title"] if "title" in df_movies else pd.Series(dtype=object)
title_by_index = (
    pd.Series(index=movieId_by_index, data=movies_map.reindex(movieId_by_index).values)
    .fillna(pd.Series([f"Item #{i}" for i in range(n_items)], index=movieId_by_index))
    .to_numpy()
)
assert len(movieId_by_index) == n_items
assert len(title_by_index)   == n_items

n_users: 138493 n_items: 26744


In [97]:
from collections import defaultdict

# Seen items from TRAIN only
seen_by_user = defaultdict(set)
for u, i in zip(train_df["user_idx"].values, train_df["movie_idx"].values):
    seen_by_user[int(u)].add(int(i))
    
REL_THRESH = 3.8
gt_by_user = defaultdict(set)
for u, i, r in zip(test_df["user_idx"].values, test_df["movie_idx"].values, test_df["rating"].values):
    if r >= REL_THRESH:
        gt_by_user[int(u)].add(int(i))


In [87]:
df_movies.movieId.unique()

array([     1,      2,      3, ..., 131258, 131260, 131262],
      shape=(27278,))

In [9]:
df_movies = pd.read_csv('movie.csv')

In [10]:
df = df_rating.merge(df_movies, on = 'movieId', how = 'left')

In [11]:
df['user_idx'] = df['userId'].map(user_dict)
df['movie_idx'] = df['movieId'].map(movie_dict)

In [12]:
df.head()

,userId,movieId,rating,timestamp,year,title,genres,user_idx,movie_idx
0,1,2,3.5,2005-04-02 23:53:47,2005,Jumanji (1995),Adventure|Children|Fantasy,0,0
1,1,29,3.5,2005-04-02 23:31:16,2005,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi,0,1
2,1,32,3.5,2005-04-02 23:33:39,2005,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,0,2
3,1,47,3.5,2005-04-02 23:32:07,2005,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,0,3
4,1,50,3.5,2005-04-02 23:29:40,2005,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,0,4


In [13]:
#temporal based split if needed, it didn't work so well in our case
# def temporal_split_per_user(df, val_frac=0.1, test_frac=0.1, min_train=1):
#     # df must include: user_idx, item_idx, rating, timestamp
#     parts = []
#     for u, g in df.groupby("user_idx", sort=False):
#         g = g.sort_values("timestamp")
#         n = len(g)
#         n_test = int(n * test_frac)
#         n_val  = int(n * val_frac)
#         n_train = n - n_val - n_test
#         if n_train < min_train:
#             # Not enough history → put all into train (or skip user)
#             parts.append((g, g.iloc[0:0], g.iloc[0:0]))
#             continue
#         g_train = g.iloc[: n_train]
#         g_val   = g.iloc[n_train: n_train + n_val]
#         g_test  = g.iloc[n_train + n_val:]
#         parts.append((g_train, g_val, g_test))
#     train = pd.concat([p[0] for p in parts], ignore_index=True)
#     val   = pd.concat([p[1] for p in parts], ignore_index=True)
#     test  = pd.concat([p[2] for p in parts], ignore_index=True)
#     return train, val, test

# train_df, val_df, test_df = temporal_split_per_user(df, val_frac=0.1, test_frac=0.1, min_train=1)

In [13]:
from sklearn.model_selection import train_test_split

In [37]:
train_df, test_df = train_test_split(df, test_size = 0.1, random_state= 10)
train_df, val_df = train_test_split(train_df, test_size = 0.1, random_state= 10)

In [19]:
def process_df(df):
    x = {'user_idx': df['user_idx'].astype(np.int32).values, 'movie_idx': df['movie_idx'].astype(np.int32).values}
    y = df['rating'].astype(np.float32).values

    ds = tf.data.Dataset.from_tensor_slices((x, y)).batch(32000).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = process_df(train_df)
val_ds = process_df(val_df)
test_ds = process_df(test_df)

I0000 00:00:1759682619.334783  872161 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5561 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [14]:
# #removing movies which never appeared in training dataset
# val_df = val_df[val_df['movie_idx'].isin(set(train_df["movie_idx"].unique()))].copy()
# test_df = test_df[test_df['movie_idx'].isin(set(train_df["movie_idx"].unique()))].copy()

In [119]:
len(df['userId'].unique())

138491

In [32]:
n_users = len(df['userId'].unique())
n_movies = len(df['movieId'].unique())

class MFModel(Model):
    def __init__(self, n_users, n_movies, dims = 128, l2 = 0.0001, init_mu = 3.52):
        super().__init__()

        self.user_emb = layers.Embedding(n_users, dims, embeddings_regularizer = regularizers.l2(l2))
        self.movie_emb = layers.Embedding(n_movies, dims, embeddings_regularizer = regularizers.l2(l2))
        
        self.user_bias = layers.Embedding(n_users, 1, embeddings_regularizer = regularizers.l2(l2))
        self.movie_bias = layers.Embedding(n_movies, 1, embeddings_regularizer = regularizers.l2(l2))

        self.global_mu = tf.Variable([init_mu], dtype = tf.float32, trainable = True)

        #NN part
        
        self.res1 = layers.Dense(64, use_bias= False)
        self.dense1a = layers.Dense(64, activation= 'relu')
        self.drop1 = layers.Dropout(0.2)
        self.dense1b = layers.Dense(64, activation= 'relu')

        self.res2 = layers.Add()
        self.dense2a = layers.Dense(dims, activation= 'relu')
        self.drop2 = layers.Dropout(0.2)
        self.dense2b = layers.Dense(dims, activation= 'relu')

        self.res3 = layers.Add()
        self.dense3a = layers.Dense(dims, activation= 'relu')
        self.drop3 = layers.Dropout(0.2)
        self.dense3b = layers.Dense(dims, activation= 'relu')
        
        self.out = layers.Dense(1)

    def call(self, inputs):
        u = inputs['user_idx']
        m = inputs['movie_idx']
        gu = self.user_emb(u)
        gm = self.movie_emb(m)

        g_out = gu*gm

        dn_u = self.user_emb(u)
        dn_m = self.movie_emb(m)
        x = tf.concat([dn_u, dn_m], axis = 1)


        #block 1
        x_res = self.res1(x)
        x = self.dense1a(x_res)
        x = self.drop1(x)
        x = self.dense1b(x)
        x = self.drop1(x)

        #block 2
        x = self.res2([x, x_res])
        x = self.dense2a(x)
        x = self.drop2(x)
        x = self.dense2b(x)
        x = self.drop2(x)


        #block 3
        x = self.res3([x, g_out])
        x = self.dense3a(x)
        x = self.drop3(x)
        x = self.dense3b(x)
        x = self.drop3(x)
        
        x = tf.concat([g_out, x], axis = 1)
        y = tf.squeeze(self.out(x), -1)
        
        bu = tf.squeeze(self.user_bias(u), axis = -1)
        bm = tf.squeeze(self.movie_bias(m), axis = -1)

        

        pred = self.global_mu + bu + bm + y

        return pred                                       
                                          
        

In [33]:
tempm1 = MFModel(n_users, n_movies)

In [34]:
_ = tempm1({"user_idx": tf.zeros([1], tf.int32),
           "movie_idx": tf.zeros([1], tf.int32)})
tempm1.summary()

Model: "mf_model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (1, 128)               │    17,727,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_9 (Embedding)         │ (1, 128)               │     3,423,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_10 (Embedding)        │ (1, 1)                 │       138,493 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_11 (Embedding)        │ (1, 1)                 │        26,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (1, 64)                │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (1, 64)                │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (1, 64)                │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ add_4 (Add)                     │ (1, 64)                │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (1, 128)               │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (1, 128)               │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ add_5 (Add)                     │ (1, 128)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (1, 128)               │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (1, 128)               │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (1, 1)                 │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,398,390 (81.63 MB)

 Trainable params: 21,398,390 (81.63 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
for i, j in train_ds.take(1):
    print(i)

{'user_idx': <tf.Tensor: shape=(32000,), dtype=int32, numpy=
array([ 22883,   3655,   3617, ...,  63193, 110865,  47909],
      shape=(32000,), dtype=int32)>, 'movie_idx': <tf.Tensor: shape=(32000,), dtype=int32, numpy=
array([  651,   533, 10759, ...,  2626,  2093,   688],
      shape=(32000,), dtype=int32)>}


2025-10-04 14:32:14.779263: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [21]:
def rmse(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, 0.5, 5.0)
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

In [37]:
l2 = 0.0
lr = 0.001
epochs = 100

model = MFModel(n_users, n_movies, dims = 256, l2 = l2)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

model.compile(
    optimizer=optimizer,
    loss="mse",
    metrics=[rmse]
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_rmse", patience=5, restore_best_weights=True, mode = 'min'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(monitor= 'val_loss', factor= 0.5, patience = 2)]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 60s 99ms/step - loss: 0.7368 - rmse: 0.8576 - val_loss: 0.6886 - val_rmse: 0.8296 - learning_rate: 0.0010
Epoch 2/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 40s 79ms/step - loss: 0.6596 - rmse: 0.8120 - val_loss: 0.6645 - val_rmse: 0.8150 - learning_rate: 0.0010
Epoch 3/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 39s 75ms/step - loss: 0.6016 - rmse: 0.7754 - val_loss: 0.6758 - val_rmse: 0.8218 - learning_rate: 0.0010
Epoch 4/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 38s 74ms/step - loss: 0.5588 - rmse: 0.7473 - val_loss: 0.7004 - val_rmse: 0.8360 - learning_rate: 0.0010
Epoch 5/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 36s 71ms/step - loss: 0.5276 - rmse: 0.7260 - val_loss: 0.6789 - val_rmse: 0.8237 - learning_rate: 5.0000e-04
Epoch 6/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 41s 81ms/step - loss: 0.5073 - rmse: 0.7118 - val_loss: 0.6880 - val_rmse: 0.8290 - learning_rate: 5.0000e-04
Epoch 7/100
507/507 ━━━━━━━━━━━━━━━━━━━━ 41s 80ms/step - loss: 0.4832 - rmse: 0.6946 - val_loss: 0.6820 - val_

In [38]:
test_metrics = model.evaluate(test_ds, return_dict=True)
print("Test metrics:", test_metrics)

63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 112ms/step - loss: 0.6641 - rmse: 0.8149
Test metrics: {'loss': 0.6641114354133606, 'rmse': 0.8148663640022278}


In [83]:
df[["userId","user_idx"]]

,userId,user_idx
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
20000258,138493,138492
20000259,138493,138492
20000260,138493,138492
20000261,138493,138492


In [34]:
def recommend_for_user(raw_user_id, topN=10, batch=200000):
    # Map raw userId -> user_idx
    row = df[["userId", "user_idx"]].drop_duplicates()
    row = row[row["userId"] == raw_user_id]
    if row.empty:
        return []  # cold-start user: handle elsewhere

    uidx = int(row["user_idx"].iloc[0])

    n_items = int(model.movie_emb.input_dim)
    
    # Score all items for this user in batches
    scores = np.empty(n_items, dtype=np.float32)
    for start in range(0, n_items, batch):
        end = min(start + batch, n_items)
        # Ensure int32 for embedding inputs
        u_batch = tf.fill([end - start], tf.cast(uidx, tf.int32))
        i_batch = tf.range(start, end, dtype=tf.int32)

        s = model({"user_idx": u_batch, "movie_idx": i_batch}, training=False).numpy()
        scores[start:end] = s

    # Mask already seen items
    seen = list(seen_by_user.get(uidx, []))
    # Keep only valid indices
    seen = [i for i in seen if 0 <= i < n_items]
    if seen:
        scores[seen] = -1e9

    # Top-N extraction
    top_idx = np.argpartition(scores, -topN)[-topN:]
    top_idx = top_idx[np.argsort(scores[top_idx])[::-1]]

    rec_movieIds = movieId_by_index[top_idx].tolist()
    rec_titles   = title_by_index[top_idx].tolist()

    return list(zip(rec_movieIds, rec_titles))


In [73]:
recommend_for_user(502, topN = 10)

[(858, 'Godfather, The (1972)'),
 (318, 'Shawshank Redemption, The (1994)'),
 (7502, 'Band of Brothers (2001)'),
 (93040, 'Civil War, The (1990)'),
 (527, "Schindler's List (1993)"),
 (912, 'Casablanca (1942)'),
 (77658, 'Cosmos (1980)'),
 (50, 'Usual Suspects, The (1995)'),
 (82143, 'Alone in the Wilderness (2004)'),
 (1148, 'Wallace & Gromit: The Wrong Trousers (1993)')]

## AutoRec

In [38]:
import numpy as np
import scipy.sparse as sp

n_users = int(df["user_idx"].max()) + 1
n_items = int(df["movie_idx"].max()) + 1
global_mean = df['rating'].mean()

def build_csr(dataframe, n_users, n_items):
    rows = dataframe["user_idx"].to_numpy(np.int32)
    cols = dataframe["movie_idx"].to_numpy(np.int32)
    vals = dataframe["rating"].to_numpy(np.float32)
    return sp.csr_matrix((vals, (rows, cols)), shape=(n_users, n_items))

train_csr = build_csr(train_df, n_users, n_items)
val_csr   = build_csr(val_df,   n_users, n_items)
test_csr  = build_csr(test_df,  n_users, n_items)

def make_user_autorec_ds(csr_mat, n_items, batch_size=512, shuffle=True):
    user_ids = np.arange(csr_mat.shape[0], dtype=np.int32)

    def gen():
        idx = user_ids.copy()
        if shuffle:
            np.random.shuffle(idx)
        for u in idx:
            row = csr_mat.getrow(u).toarray().astype(np.float32).squeeze(0) 
            yield row, row #(x,y)

    spec = (
        tf.TensorSpec(shape=(n_items,), dtype=tf.float32),
        tf.TensorSpec(shape=(n_items,), dtype=tf.float32),
    )
    ds = tf.data.Dataset.from_generator(gen, output_signature=spec)
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)


def center_csr_global(csr, mean):
    # Returns a new CSR with values shifted by -mean, adding this here seperately, can be commented to remove if dont want centered
    return sp.csr_matrix((csr.data - mean, csr.indices, csr.indptr), shape=csr.shape)

train_csr = center_csr_global(train_csr, global_mean)
val_csr   = center_csr_global(val_csr,   global_mean)
test_csr  = center_csr_global(test_csr,  global_mean)


train_ds = make_user_autorec_ds(train_csr, n_items, 512, True)
val_ds   = make_user_autorec_ds(val_csr,   n_items, 512, False)
test_ds  = make_user_autorec_ds(test_csr,  n_items, 512, False)


In [40]:
inputs = layers.Input(shape=(n_items,), dtype=tf.float32)

x = layers.Dropout(0.6)(inputs)  
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)

out = layers.Dense(n_items, activation=None)(x)
model_auto = tf.keras.Model(inputs, out)

def masked_mse(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, 0.0), tf.float32)
    diff2 = tf.square(y_pred - y_true) * mask
    return tf.reduce_sum(diff2) / (tf.reduce_sum(mask) + 1e-8)

class MaskedRMSE(tf.keras.metrics.Metric):
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.se = self.add_weight(name="se", shape=(), initializer="zeros", dtype=tf.float32)
        self.w  = self.add_weight(name="w",  shape=(), initializer="zeros", dtype=tf.float32)
    def update_state(self, y_true, y_pred, sample_weight=None):
        mask = tf.cast(tf.not_equal(y_true, 0.0), tf.float32)
        diff2 = tf.square(y_pred - y_true) * mask
        self.se.assign_add(tf.reduce_sum(diff2))
        self.w.assign_add(tf.reduce_sum(mask))
    def result(self):
        return tf.sqrt(self.se / (self.w + 1e-8))
    def reset_states(self):
        self.se.assign(0.0); self.w.assign(0.0)


In [41]:
model_auto.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 26717)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 26717)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │    13,679,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 26717)          │     3,446,493 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,290,333 (65.96 MB)

 Trainable params: 17,290,333 (65.96 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
model_auto.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                   loss=masked_mse,
                   metrics=[MaskedRMSE()])

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_rmse", mode="min", patience=4, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_rmse", mode="min", factor=0.5, patience=2, min_lr=1e-5),
]

history = model_auto.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=callbacks, verbose=1)

Epoch 1/50


2025-10-05 23:29:17.079346: I external/local_xla/xla/service/service.cc:163] XLA service 0x55bcf6f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-05 23:29:17.079586: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-10-05 23:29:17.158462: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-05 23:29:17.461260: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 90501
2025-10-05 23:29:18.449544: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_507', 108 bytes spill stores, 108 bytes spill loads

2025-10-05 23:29:18.932904: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346

      1/Unknown 6s 6s/step - loss: 1.1709 - rmse: 1.0821

I0000 00:00:1759687161.607921  872458 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


    270/Unknown 60s 203ms/step - loss: 0.9161 - rmse: 0.9561

...

2025-10-05 23:47:33.589476: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 4817683161425880201
2025-10-05 23:47:33.589545: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10799239012569361700


271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - loss: 0.4454 - rmse: 0.6670

2025-10-05 23:48:28.063929: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 4817683161425880201
2025-10-05 23:48:28.064058: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10799239012569361700


271/271 ━━━━━━━━━━━━━━━━━━━━ 108s 397ms/step - loss: 0.4493 - rmse: 0.6699 - val_loss: 0.4043 - val_rmse: 0.6361 - learning_rate: 5.0000e-04


2025-10-05 23:49:21.148146: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 4817683161425880201
2025-10-05 23:49:21.148239: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10799239012569361700


In [45]:
test_metrics = model_auto.evaluate(test_ds, verbose=1)
print(test_metrics)

271/271 ━━━━━━━━━━━━━━━━━━━━ 55s 202ms/step - loss: 0.3769 - rmse: 0.6144
[0.3768567442893982, 0.6144348382949829]


2025-10-05 23:52:11.944856: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 4817683161425880201
2025-10-05 23:52:11.944943: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10799239012569361700
/home/anubh/miniconda3/envs/dl-env/lib/python3.11/site-packages/keras/src/trainers/epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


## MF-BPR

In [107]:
REL_THRESH = THRESH 
gt_by_user = defaultdict(set)
for u, i, r in zip(test_df["user_idx"].values,
                   test_df["movie_idx"].values,
                   test_df["rating"].values):
    if r >= REL_THRESH:
        gt_by_user[int(u)].add(int(i))

# Warm-user eval cohort: users with TRAIN history and at least 1 GT item
warm_users = set(np.unique(train_df["user_idx"].values).tolist())
users_eval = np.array([u for u in np.unique(test_df["user_idx"].values)
                       if (u in warm_users) and (len(gt_by_user.get(int(u), ())) > 0)],
                      dtype=np.int32)

d = 128  # embedding dimension 
rng = np.random.default_rng(42)

user_factors = tf.Variable(tf.random.normal([n_users, d], stddev=0.05))
item_factors = tf.Variable(tf.random.normal([n_items, d], stddev=0.05))
item_bias    = tf.Variable(tf.zeros([n_items], tf.float32))  # optional

def sample_triplets(batch_size):
    u_batch = np.empty(batch_size, np.int32)
    i_batch = np.empty(batch_size, np.int32)
    j_batch = np.empty(batch_size, np.int32)

    # sample users with positives
    ub = users_with_pos[rng.integers(0, len(users_with_pos), size=batch_size)]
    u_batch[:] = ub

    # sample a positive per user
    for k, u in enumerate(ub):
        pos_list = list(positives_by_user[int(u)])
        i_batch[k] = pos_list[rng.integers(0, len(pos_list))]

    # sample negatives (resample on collision)
    j_batch = rng.integers(0, n_items, size=batch_size, dtype=np.int32)
    for k, u in enumerate(ub):
        while j_batch[k] in positives_by_user[int(u)]:
            j_batch[k] = int(rng.integers(0, n_items))
    return u_batch, i_batch, j_batch

In [110]:
item_factors

<tf.Variable 'Variable:0' shape=(26744, 128) dtype=float32, numpy=
array([[-0.00320732, -0.06844097,  0.12959965, ...,  0.02854785,
        -0.01091599,  0.04180642],
       [-0.06230679, -0.022381  , -0.02652861, ..., -0.02634893,
         0.01627517,  0.01918394],
       [ 0.01496974,  0.01557859, -0.02968986, ...,  0.06741958,
        -0.05334706,  0.09160335],
       ...,
       [-0.02817411,  0.0003989 ,  0.00116769, ...,  0.01579854,
         0.06870127, -0.02801405],
       [-0.03164991,  0.04216761, -0.06880376, ...,  0.003628  ,
        -0.05193505,  0.01366765],
       [ 0.00552381, -0.06155177,  0.04973213, ..., -0.01202566,
        -0.013162  , -0.02437686]], shape=(26744, 128), dtype=float32)>

In [112]:
opt = tf.keras.optimizers.Adam(learning_rate=3e-3)
l2 = 1e-6
clip_norm = 5.0

@tf.function
def train_step(u, i, j):
    with tf.GradientTape() as tape:
        U  = tf.gather(user_factors, u)    # [B, d]
        Vi = tf.gather(item_factors, i)    # [B, d]
        Vj = tf.gather(item_factors, j)    # [B, d]
        bi = tf.gather(item_bias, i)       # [B]
        bj = tf.gather(item_bias, j)       # [B]

        s_pos = tf.reduce_sum(U * Vi, axis=1) + bi
        s_neg = tf.reduce_sum(U * Vj, axis=1) + bj
        x = s_pos - s_neg

        bpr = - tf.reduce_mean(tf.math.log_sigmoid(x))   # pairwise ranking loss
        reg = l2 * (tf.nn.l2_loss(U) + tf.nn.l2_loss(Vi) + tf.nn.l2_loss(Vj) +
                    tf.nn.l2_loss(bi) + tf.nn.l2_loss(bj))
        loss = bpr + reg

    vars_ = [user_factors, item_factors, item_bias]
    grads = tape.gradient(loss, vars_)
    grads, _ = tf.clip_by_global_norm(grads, clip_norm)
    opt.apply_gradients(zip(grads, vars_))
    return loss

def bpr_scores_batch(u_batch):
    U = tf.gather(user_factors, u_batch)                         # [B, d]
    scores = tf.matmul(U, item_factors, transpose_b=True) + item_bias  # [B, n_items]
    return scores.numpy()

def topk_batch(scores, K, batch_users):
    # scores: [B, n_items]
    scores = scores.copy()
    for bi, u in enumerate(batch_users):
        seen = seen_by_user.get(int(u), None)
        if seen:
            scores[bi, list(seen)] = -1e9
    idx = np.argpartition(scores, -K, axis=1)[:, -K:]
    row = np.arange(idx.shape[0])[:, None]
    idx_sorted = idx[row, np.argsort(scores[row, idx], axis=1)[:, ::-1]]
    return idx_sorted  # [B, K]

def precision_at_k_batched(K=10, batch_users=1024):
    precisions = []
    for s in range(0, len(users_eval), batch_users):
        ub = users_eval[s:s+batch_users]
        scores = bpr_scores_batch(ub)
        recs = topk_batch(scores, K, ub)
        for bi, u in enumerate(ub):
            gt = gt_by_user.get(int(u), None)
            if not gt: continue
            hits = len(set(recs[bi]) & gt)
            precisions.append(hits / K)
    return float(np.mean(precisions)) if precisions else 0.0


In [113]:
steps_per_epoch = 2000
batch_size      = 8192
epochs          = 15

for epoch in range(1, epochs + 1):
    losses = []
    for _ in range(steps_per_epoch):
        u, i, j = sample_triplets(batch_size)
        loss = train_step(u, i, j)
        losses.append(float(loss))
    p10 = precision_at_k_batched(K=10, batch_users=1024) 
    print(f"Epoch {epoch}: avg loss={np.mean(losses):.4f}  Prec@10={p10:.4f}")


Epoch 1: avg loss=0.1477  Prec@10=0.0790
Epoch 2: avg loss=0.0868  Prec@10=0.0836
Epoch 3: avg loss=0.0838  Prec@10=0.0870
Epoch 4: avg loss=0.0828  Prec@10=0.0893
Epoch 5: avg loss=0.0823  Prec@10=0.0894
Epoch 6: avg loss=0.0821  Prec@10=0.0916
Epoch 7: avg loss=0.0819  Prec@10=0.0906
Epoch 8: avg loss=0.0818  Prec@10=0.0933
Epoch 9: avg loss=0.0818  Prec@10=0.0924
Epoch 10: avg loss=0.0817  Prec@10=0.0910
Epoch 11: avg loss=0.0816  Prec@10=0.0924
Epoch 12: avg loss=0.0817  Prec@10=0.0923
Epoch 13: avg loss=0.0816  Prec@10=0.0923
Epoch 14: avg loss=0.0815  Prec@10=0.0938
Epoch 15: avg loss=0.0816  Prec@10=0.0926


In [126]:
# ---- MF+BPR scorer (uses trained embeddings) ----
def bpr_scores_batch(u_batch):
    U = tf.gather(user_factors, u_batch)                         # [B, d]
    scores = tf.matmul(U, item_factors, transpose_b=True)        # [B, n_items]
    scores = scores + item_bias                                  # broadcast add
    return scores.numpy()

# ---- AutoRec scorer (uses your trained AE and TRAIN CSR rows as input) ----
# If you trained with global centering, you can add global_mean back. For ranking it doesn't change order.
def autorec_scores_batch(u_batch, add_back_mean=0.0):
    Xb = train_csr[u_batch].toarray().astype(np.float32)         # [B, n_items]
    preds = model_auto(Xb, training=False).numpy()                # [B, n_items]
    if add_back_mean:
        preds = preds + float(add_back_mean)
    return preds


def recall_at_k(users, K, scores_fn, batch_users=1024):
    users = np.asarray(users, dtype=np.int32)
    recalls = []
    bad_seen_total = 0
    for s in range(0, len(users), batch_users):
        ub = users[s:s+batch_users]
        scores = scores_fn(ub)                                    # [B, n_items]
        n_cols = scores.shape[1]

        # Safe masking with guard + lightweight diagnostics
        for bi, u in enumerate(ub):
            seen = seen_by_user.get(int(u), None)
            if seen:
                cols = np.fromiter(seen, dtype=np.int64)
                bad = (cols < 0) | (cols >= n_cols)
                bad_seen_total += int(bad.sum())
                cols = cols[~bad]
                if cols.size:
                    scores[bi, cols] = -1e9

        idx = np.argpartition(scores, -K, axis=1)[:, -K:]
        row = np.arange(idx.shape[0])[:, None]
        idx_sorted = idx[row, np.argsort(scores[row, idx], axis=1)[:, ::-1]]

        for bi, u in enumerate(ub):
            gt = gt_by_user.get(int(u), None)
            if not gt:
                continue
            hits = len(set(idx_sorted[bi]) & gt)
            recalls.append(hits / len(gt))

    if bad_seen_total > 0:
        print(f"[warn] Ignored {bad_seen_total} out-of-range seen indices during masking.")
    return float(np.mean(recalls)) if recalls else 0.0



In [127]:
warm_users = set(np.unique(train_df["user_idx"].values).tolist())

users_eval = np.array([u for u in np.unique(test_df["user_idx"].values)
                       if (u in warm_users) and (len(gt_by_user.get(int(u), ())) > 0)],
                      dtype=np.int32)

recall10_bpr   = recall_at_k(users_eval, K=10, scores_fn=bpr_scores_batch)
recall10_ae    = recall_at_k(users_eval, K=10, scores_fn=lambda ub: autorec_scores_batch(ub, add_back_mean=global_mean))
print("Recall@10 (BPR):", recall10_bpr)
print("Recall@10 (AutoRec):", recall10_ae)

[warn] Ignored 24 out-of-range seen indices during masking.
Recall@10 (BPR): 0.1645596539634774
Recall@10 (AutoRec): 6.800261421028686e-05


In [122]:

def get_user_idx(raw_user_id):
    row = df.loc[df["userId"].eq(raw_user_id), ["user_idx"]].drop_duplicates()
    if row.empty:
        return None
    return int(row["user_idx"].iloc[0])

def recommend_top10_bpr(raw_user_id):
    uidx = get_user_idx(raw_user_id)
    if uidx is None:
        return []  # cold-start; fall back to popularity
    scores = bpr_scores_batch(np.array([uidx]))[0]               # [n_items]
    seen = seen_by_user.get(uidx, None)
    if seen:
        scores[list(seen)] = -1e9
    idx = np.argpartition(scores, -10)[-10:]
    idx = idx[np.argsort(scores[idx])[::-1]]
    return [(int(movieId_by_index[i]), str(title_by_index[i]), float(scores[i])) for i in idx]

def recommend_top10_autorec(raw_user_id, add_back_mean=0.0):
    uidx = get_user_idx(raw_user_id)
    if uidx is None:
        return []
    scores = autorec_scores_batch(np.array([uidx]), add_back_mean=add_back_mean)[0]
    seen = seen_by_user.get(uidx, None)
    if seen:
        scores[list(seen)] = -1e9
    idx = np.argpartition(scores, -10)[-10:]
    idx = idx[np.argsort(scores[idx])[::-1]]
    return [(int(movieId_by_index[i]), str(title_by_index[i]), float(scores[i])) for i in idx]


In [128]:
print("Top‑10 (BPR):", recommend_top10_bpr(raw_user_id=502))
print("\nTop‑10 (AutoRec):", recommend_top10_autorec(raw_user_id=502, add_back_mean=global_mean))

Top‑10 (BPR): [(912, 'Casablanca (1942)', 4.272552490234375), (1284, 'Big Sleep, The (1946)', 4.100092887878418), (1283, 'High Noon (1952)', 4.0522308349609375), (913, 'Maltese Falcon, The (1941)', 3.9904019832611084), (1304, 'Butch Cassidy and the Sundance Kid (1969)', 3.9713573455810547), (969, 'African Queen, The (1951)', 3.9665236473083496), (930, 'Notorious (1946)', 3.946599245071411), (904, 'Rear Window (1954)', 3.929795742034912), (951, 'His Girl Friday (1940)', 3.8872950077056885), (1953, 'French Connection, The (1971)', 3.85772705078125)]

Top‑10 (AutoRec): [(88059, 'Bikini Summer (1991)', 5.306845664978027), (67361, 'Echelon Conspiracy (2009)', 5.292651176452637), (47028, "Sione's Wedding (Samoan Wedding) (2006)", 5.229885578155518), (104041, 'Arrival II (1998)', 5.193790435791016), (90114, 'I Dream Too Much (1935)', 5.171387672424316), (51277, '36th Chamber of Shaolin, The (Shao Lin san shi liu fang) (Master Killer) (1978)', 5.153675079345703), (58904, 'Chan Is Missing (1982

## Conclusion

In this project on the Movielens 20 Million dataset we first focused on the ratings and focused on RMSE based models, and then focused on models used for better retrievers. While all of them could be improved by using genre, tags, and more data, these models represents building blocks of modern recommendation systems.

Multi-VAE and BPR based models beside others could be used in getting better top k recommendations while RMSE based models like NeuralMF and AutoRec would focus on the better score creation. And while we did obtain a good score in AutoRec, NeuralMF is also a valid good choice as its more robust in our testing. But for retrieving task MF-BPR outperformed both of them who were just recommending top k movies randomly. Nevertheless, all three models have their pros and cons and we could use a stack combining features of all of them, making a producing ready and scalable Recommendation System.